In [28]:
import random
import numpy as np

# 1. Criação do grid

In [29]:
def randomCell():
  line = random.randint(0, 7)
  col = random.randint(0, 7)
  return [line, col]

In [30]:
def gridInit():

  grid = []
  currentLineList = []
  lines = 8
  cols = 8
  currentLine = 1
  currentCol = 1

  # percorre linha por linha
  while currentLine <= lines:
    while currentCol <= cols:
      currentLineList.append({
          "valueUp": 0,
          "valueDown": 0,
          "valueLeft": 0,
          "valueRight": 0,
          "type": "-"
          })
      currentCol += 1
    grid.append(currentLineList)
    currentLineList = []
    currentLine += 1
    currentCol = 1 # reseta a coluna pro início cada vez que a linha muda

  # colocando célular randômicas
  randomStart = randomCell()
  randomEnd = randomCell()
  randomMountain = randomCell()
  randomQuicksand = randomCell()
  grid[randomStart[0]][randomStart[1]]["type"] = "S"
  grid[randomEnd[0]][randomEnd[1]]["type"] = "E"
  grid[randomMountain[0]][randomMountain[1]]["type"] = "M"
  grid[randomQuicksand[0]][randomQuicksand[1]]["type"] = "Q"

  return grid, randomStart, randomEnd, randomMountain, randomQuicksand

# 2. Print do grid

In [31]:
def gridPrint(grid):
  currentLine = 0
  currentCol = 0
  while currentLine < 8:
    strLine = ""
    while currentCol < 8:
      strLine += grid[currentLine][currentCol]["type"] + " "
      currentCol += 1
    print(strLine)
    currentCol = 0
    currentLine += 1

# 3. Conjunto de ações que o agente pode realizar
O ambiente assume que o agente não pode tomar a decisão de ir para fora do grid ou mover-se para uma posição de montanha. Essas ações não estarão disponíveis para escolha dependendo de sua posição.
Se, por acaso, o número de montanhas aumentar, a função `availableActions()` contempla + de uma montanha por grid.
A função não contempla grids diferentes de 8x8, o código teria que ser editado para isso.

In [32]:
def availableActions(agentPosition, mountainPosition):
  # positions têm forma [line, col]
  up = True
  down = True
  left = True
  right = True

  # handle de linhas e colunas limites
  if agentPosition[1] == 7: # se está na col 7
    right = False
  elif agentPosition[1] == 0: # se está na col 0
    left = False
  if agentPosition[0] == 0: # se está na line 0
    up = False
  elif agentPosition[0] == 7:
    down = False

  # handle de mountain
  mountainLine = mountainPosition[0]
  mountainCol = mountainPosition[1]
  if agentPosition[0] + 1 == mountainLine: # se a montanha está em baixo do agente
    down = False
  if agentPosition[0] - 1 == mountainLine: # se a montanha está em cima do agente
    up = False
  if agentPosition[1] + 1 == mountainCol: # se a montanha está à direita do agente
    right = False
  if agentPosition[1] - 1 == mountainCol: # se a montanha está à esquerda do agente
    left = False

  return up, down, left, right

# 4. Recompensas
- Entrar na areia movediça = -10
- Mover-se para qualquer posição que não seja final = -1
- Mover-se para estado final = +100


In [33]:
def rewardBasedOnAction(action, agentPosition, randomEnd, randomQuicksand):
  if action == 0: # up
    newPosition = agentPosition[0] - 1
  elif action == 1: # down
    newPosition = agentPosition[0] + 1
  elif action == 2: # left
    newPosition = agentPosition[1] - 1
  elif action == 3: # right
    newPosition = agentPosition[1] + 1
  else:
    raise ValueError("Action out of range")

  if newPosition == randomEnd:
    return newPosition, +100
  elif newPosition == randomQuicksand:
    return newPosition, -10
  else:
    return newPosition, -1

# Atualização de qualidades com Q-learning
O agente tentará aprender a melhor política utilizando o algoritmo Q-learning

In [34]:
def updateCellQuality(oldQuality, reward, maxNextActionValue):
  # parâmetros do q-learning
  discountFactor = 0.9
  learningRate = 0.1
  return oldQuality + (learningRate * (reward + (discountFactor * maxNextActionValue) - oldQuality))

In [35]:
def actionChooser(grid, up, down, left, right):
  exploitationRate = 60 # em %
  whatWillIDo = random.randint(1, 100)
  if whatWillIDo >= exploitationRate: # explora
    isRandomActionPossible = False
    # fica sorteando uma ação enquanto a sorteada não é possível com base nos parâmetros booleanos da função
    while isRandomActionPossible == False:
      action = random.randint(0, 3)
      if action == 0:
        if up == False:
          pass
        else:
          isRandomActionPossible = True
      elif action == 1:
        if down == False:
          pass
        else:
          isRandomActionPossible = True
      elif action == 2:
        if left == False:
          pass
        else:
          isRandomActionPossible = True
      elif action == 3:
        if right == False:
          pass
        else:
          isRandomActionPossible = True

  else: # exploita
    # seleciona a maior qualidade de estado/ação


  return action

In [37]:
def maxNextStateActionQualityCalculator(grid, nextPosition):
  goingUp = grid[nextPosition[0]][nextPosition[1]]["valueUp"]
  goingDown = grid[nextPosition[0]][nextPosition[1]]["valueDown"]
  goingLeft = grid[nextPosition[0]][nextPosition[1]]["valueLeft"]
  goingRight = grid[nextPosition[0]][nextPosition[1]]["valueRight"]
  allValues = [goingUp, goingDown, goingLeft, goingRight]
  return np.argmax(allValues)

# Execução do ambiente

In [38]:
# inicialização do ambiente
grid, randomStart, endPosition, mountainPosition, quicksandPosition = gridInit()
gridPrint(grid)

- - - - - - - - 
- - - - - - - - 
- - - - - - - - 
- - - - - Q S - 
- - - - - - - E 
- - - - - - - - 
- - - - - - - M 
- - - - - - - - 


## Rodando Q-learning

In [39]:
def turnActionToString(action):
  if action == 0:
    return "valueUp"
  elif action == 1:
    return "valueDown"
  elif action == 2:
    return "valueLeft"
  elif action == 3:
    return "valueRight"

In [40]:
agentPosition = randomStart
# actions up, down, left, right -> 0, 1, 2, 3

iterations = 100
qualityTable = []

for i in iterations:
  # finds the available actions for the current position
  up, down, left, right = availableActions(agentPosition, randomMountain)

  # chooses hte action based
  action = actionChooser(grid, up, down, left, right)
  nextPosition, reward = rewardBasedOnAction()
  maxNextStateActionQuality = maxNextStateActionQualityCalculator(grid, nextPosition)

  # utiliza as informações para atualizar as qualidades das qualidades
  actionString = turnActionToString(action)
  currentStateActionQuality = grid[agentPosition[0]][agentPosition[1]][actionString]
  updatedStateActionQuality = updateCellQuality(currentStateActionQuality)
  grid[agentPosition[0]][agentPosition[1]][actionString] = updatedStateActionQuality

  agentPosition = nextPosition